In [20]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from google.colab import drive
drive.mount('/content/drive')


obcine = pd.read_csv("/content/drive/MyDrive/napoved_obcine/prebivalstvo_zdruzeno.csv")
regije = pd.read_csv("/content/drive/MyDrive/napoved_obcine/podatkiRegije_zdruzeno.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
print(obcine.head())

       obcina  leto  Gross income - TOTAL  Income from work  \
0  Ajdovščina  2014              10110.67           7193.91   
1  Ajdovščina  2015              10246.16           7402.73   
2  Ajdovščina  2016              10640.96           7778.48   
3  Ajdovščina  2017              11152.95           8213.25   
4  Ajdovščina  2018              11949.15           8865.33   

  Parental, family and social benefits Pensions  \
0                               610.26  1998.46   
1                               579.02  2003.56   
2                               578.57  2012.99   
3                               582.25  2056.81   
4                               590.45  2108.46   

  Property, capital and other income Basic or less Education - TOTAL Tertiary  \
0                             308.06          4870             15765     2943   
1                             260.86          4759             15798     3066   
2                             270.93          4635             15871   

In [22]:
# Za regije
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from scipy.stats import linregress

warnings.filterwarnings("ignore")

# --- Glavna funkcija za napovedovanje z upoštevanjem trendov ---
def predict_with_trends(regije, future_years=[2025, 2026, 2027]):
    # 1. Predprocesiranje podatkov
    regije = regije.replace(['...', ''], np.nan)
    numeric_cols = [col for col in regije.select_dtypes(include=np.number).columns if col != 'leto']

    for col in numeric_cols:
        regije[col] = pd.to_numeric(regije[col], errors='coerce')

    # 2. Izračun trendov za vsako regijo in spremenljivko
    predictions = []
    current_data = regije[regije['leto'] < 2025].copy()

    for regija in current_data['regija'].unique():
        regija_data = current_data[current_data['regija'] == regija].copy()

        for col in numeric_cols:
            # Filtriranje manjkajočih vrednosti
            clean_data = regija_data[['leto', col]].dropna()

            if len(clean_data) < 2:
                continue

            # Izračun trenda
            x = clean_data['leto']
            y = clean_data[col]

            # Linearna regresija za osnovno napoved
            model = LinearRegression()
            model.fit(x.values.reshape(-1, 1), y)

            # Izračun relativne rasti
            growth_rates = y.pct_change().dropna()
            avg_growth = growth_rates.mean() if len(growth_rates) > 0 else 0

            # Napoved za prihodnja leta
            for year in future_years:
                # Osnovna napoved
                base_pred = model.predict([[year]])[0]

                # Prilagoditev glede na trend rasti
                if 'income' in col.lower() and avg_growth > 0:
                    years_diff = year - x.max()
                    adjusted_pred = base_pred * (1 + avg_growth) ** years_diff
                else:
                    adjusted_pred = base_pred

                predictions.append({
                    'regija': regija,
                    'leto': year,
                    col: adjusted_pred
                })

    # 3. Oblikovanje rezultatov
    if predictions:
        future_df = pd.DataFrame(predictions)
        future_df = future_df.groupby(['regija', 'leto']).mean().reset_index()
        final_df = pd.concat([regije, future_df], ignore_index=True)
        return final_df.sort_values(['regija', 'leto'])
    return regije

# --- Uporaba funkcije ---
try:
    # Predpostavka: 'regije' DataFrame je že naložen
    final_results = predict_with_trends(regije)

    # Shranjevanje rezultatov
    output_file = 'Regije_z_napovedjo_trendi.csv'
    final_results.to_csv(output_file, index=False)
    print(f"Datoteka '{output_file}' uspešno ustvarjena.")

    # Analiza rezultatov
    if 'income' in final_results.columns:
        income_2023 = final_results[final_results['leto'] == 2023]['income'].mean()
        income_2027 = final_results[final_results['leto'] == 2027]['income'].mean()
        growth = ((income_2027 - income_2023) / income_2023) * 100
        print(f"Predvidena rast dohodka (2023-2027): {growth:.2f}%")

except Exception as e:
    print(f"Napaka pri analizi: {str(e)}")

Datoteka 'Regije_z_napovedjo_trendi.csv' uspešno ustvarjena.


In [23]:
# Za obine
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore")

def pripravi_podatke(df):
    """Izboljšano čiščenje podatkov"""
    df = df.replace(['z', '', '...', 'NaN', 'NA'], np.nan)

    # Pretvorba v številke za vse numerične stolpce
    numeric_cols = [col for col in df.columns if col not in ['obcina', 'leto']]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def napovedi_za_obcino(obcina_data, future_years):
    """Napoved za posamezno občino z bolj robustnim pristopom"""
    results = []
    numeric_cols = [col for col in obcina_data.columns if col not in ['obcina', 'leto']]

    # Če ni dovolj podatkov, uporabimo povprečje vseh občin
    if len(obcina_data) < 2:
        global_avg = obcina_data[numeric_cols].mean()
        for year in future_years:
            new_row = {'obcina': obcina_data['obcina'].iloc[0], 'leto': year}
            for col in numeric_cols:
                new_row[col] = round(global_avg[col], 2) if not pd.isna(global_avg[col]) else 0
            results.append(new_row)
        return results

    # Napoved za vsak stolpec posebej
    for col in numeric_cols:
        data = obcina_data[['leto', col]].dropna()

        # Če manjkajo podatki, uporabimo preprosto interpolacijo
        if len(data) < 2:
            # Poskusimo interpolirati manjkajoče vrednosti
            temp_data = obcina_data[['leto', col]].copy()
            temp_data[col] = temp_data[col].interpolate(method='linear', limit_direction='both')
            temp_data = temp_data.dropna()

            if len(temp_data) >= 2:
                data = temp_data
            else:
                # Če interpolacija ne deluje, uporabimo povprečje vseh občin
                global_avg = obcina_data[col].mean()
                pred_value = round(global_avg, 2) if not pd.isna(global_avg) else 0
                for year in future_years:
                    new_row = {'obcina': obcina_data['obcina'].iloc[0], 'leto': year}
                    for c in numeric_cols:
                        new_row[c] = pred_value if c == col else np.nan
                    results.append(new_row)
                continue

        X = data['leto'].values.reshape(-1, 1)
        y = data[col].values

        model = LinearRegression()
        model.fit(X, y)

        future_pred = model.predict(np.array(future_years).reshape(-1, 1))

        for i, year in enumerate(future_years):
            # Poskrbimo, da vsaka vrstica vsebuje vse stolpce
            if i >= len(results):
                new_row = {'obcina': obcina_data['obcina'].iloc[0], 'leto': year}
                for c in numeric_cols:
                    new_row[c] = np.nan
                results.append(new_row)

            # Access the prediction for the current year correctly
            results[i][col] = round(future_pred[i], 2)

    return results

def main():
    # Nalaganje podatkov (uporaba obstoječega DataFrame 'obcine')
    try:
        df = obcine.copy()
    except NameError:
        print("Napaka: Spremenljivka 'obcine' ne obstaja")
        return

    # Priprava podatkov
    df_clean = pripravi_podatke(df)

    # Napovedi za prihodnja leta
    future_years = [2025, 2026, 2027]
    all_predictions = []

    # Napovedi za vse občine
    for obcina in df_clean['obcina'].unique():
        obcina_data = df_clean[df_clean['obcina'] == obcina]
        predictions = napovedi_za_obcino(obcina_data, future_years)
        all_predictions.extend(predictions)

    # Združitev originalnih podatkov in napovedi
    if all_predictions:
        # Ustvarjanje DataFrame iz napovedi
        predictions_df = pd.DataFrame(all_predictions)

        # Združitev z originalnimi podatki
        combined_df = pd.concat([df_clean, predictions_df], ignore_index=True)

        # Nadomestimo morebitne NaN vrednosti s povprečjem stolpca
        numeric_cols = [col for col in df_clean.columns if col not in ['obcina', 'leto']]
        imputer = SimpleImputer(strategy='mean')
        combined_df[numeric_cols] = imputer.fit_transform(combined_df[numeric_cols])

        # Zaokrožimo vrednosti
        for col in numeric_cols:
            combined_df[col] = combined_df[col].round(2)

        # Sortiranje po občini in letu
        combined_df = combined_df.sort_values(['obcina', 'leto'])

        # Shranjevanje celotnega DataFrame
        combined_df.to_csv('obcine_z_napovedmi.csv', index=False)
        print("Podatki z napovedmi shranjeni v 'obcine_z_napovedmi.csv'")
    else:
        print("Ni bilo mogoče ustvariti napovedi")

if __name__ == "__main__":
    main()

Podatki z napovedmi shranjeni v 'obcine_z_napovedmi.csv'


In [24]:
# prompt: neredi kopijo obcine in obcine_z_napovedmi.csv, sedaj združi tabeli tako, da vzameš osnovo iz občine in k njej dodaš vse podatke iz obcine_z_napovedmi.csv, ki vrstica vsebuje letnice 2025, 2026, 2027. Shrani nov csv

import pandas as pd
import numpy as np
# Naloži obstoječo tabelo 'obcine' (to je vaša osnovna tabela)
# Predpostavka: 'obcine' DataFrame je že naložen iz prejšnjega dela kode

# Naloži tabelo z napovedmi
try:
    obcine_z_napovedmi = pd.read_csv('obcine_z_napovedmi.csv')
except FileNotFoundError:
    print("Napaka: Datoteka 'obcine_z_napovedmi.csv' ne obstaja. Prosimo, najprej jo ustvarite z zagonom prejšnjega dela kode.")
    exit()

# Filtriraj vrstice iz 'obcine_z_napovedmi', ki ustrezajo letom 2025, 2026, 2027
napovedi_za_zdruzitev = obcine_z_napovedmi[
    obcine_z_napovedmi['leto'].isin([2025, 2026, 2027])
].copy()

# Poskrbimo, da imamo enake stolpce v obeh DataFramih, preden ju združimo
# Identificiramo stolpce, ki so skupni obema in tiste, ki so samo v napovedih
common_cols = list(set(obcine.columns) & set(napovedi_za_zdruzitev.columns))
napovedi_only_cols = list(set(napovedi_za_zdruzitev.columns) - set(obcine.columns))

# V osnovni tabeli 'obcine' dodamo manjkajoče stolpce iz napovedi in jih napolnimo z NaN
for col in napovedi_only_cols:
    if col not in obcine.columns: # Preverimo, da ne dodamo stolpca, ki morda že obstaja (čeprav redko)
      obcine[col] = np.nan

# V tabeli z napovedmi poskrbimo, da so prisotni vsi skupni stolpci in morebitni novi iz napovedi
# To je že zagotovljeno z zgornjo filtracijo in izbiro stolpcev

# Sedaj združimo obe tabeli. Uporabimo pd.concat, saj dodajamo vrstice (napovedi) k obstoječim podatkom.
# Pred tem poskrbimo, da imata oba DataFrame enake stolpce v enakem vrstnem redu
all_cols = list(obcine.columns) # Uporabimo stolpce iz originalne tabele kot referenco

# Poskrbimo, da so vsi stolpci iz napovedi vključeni, tudi če jih ni v originalni tabeli
for col in napovedi_za_zdruzitev.columns:
    if col not in all_cols:
        all_cols.append(col)

# Prerazporedimo stolpce v obeh DataFrameih, da se ujemajo
obcine_reindexed = obcine.reindex(columns=all_cols)
napovedi_za_zdruzitev_reindexed = napovedi_za_zdruzitev.reindex(columns=all_cols)

# Združitev
zdruzena_tabela = pd.concat([obcine_reindexed, napovedi_za_zdruzitev_reindexed], ignore_index=True)

# Sortiranje po občini in letu za boljši pregled
zdruzena_tabela = zdruzena_tabela.sort_values(['obcina', 'leto']).reset_index(drop=True)

# Shranjevanje nove združene tabele
output_filename = 'obcine_zdruzene_z_napovedmi_2025_2027.csv'
zdruzena_tabela.to_csv(output_filename, index=False)

print(f"Nova združena tabela shranjena v '{output_filename}'")

# Prikaz prvih nekaj vrstic združene tabele
print("\nPrvih nekaj vrstic združene tabele:")
print(zdruzena_tabela.head())

# Prikaz zadnjih nekaj vrstic (da vidimo napovedi)
print("\nZadnjih nekaj vrstic združene tabele (z napovedmi za 2025-2027):")
print(zdruzena_tabela.tail())

Nova združena tabela shranjena v 'obcine_zdruzene_z_napovedmi_2025_2027.csv'

Prvih nekaj vrstic združene tabele:
       obcina  leto  Gross income - TOTAL  Income from work  \
0  Ajdovščina  1995                   NaN               NaN   
1  Ajdovščina  1996                   NaN               NaN   
2  Ajdovščina  1997                   NaN               NaN   
3  Ajdovščina  1998                   NaN               NaN   
4  Ajdovščina  1999                   NaN               NaN   

  Parental, family and social benefits Pensions  \
0                                  NaN      NaN   
1                                  NaN      NaN   
2                                  NaN      NaN   
3                                  NaN      NaN   
4                                  NaN      NaN   

  Property, capital and other income Basic or less Education - TOTAL Tertiary  \
0                                NaN           NaN               NaN      NaN   
1                                NaN  